Notes from Stat 849 at UW-Madison: http://www.stat.wisc.edu/courses/st849-bates/lectures/

## Chapter 1: The Gaussian Linear Model
Reponses: Vector-valued random variable, $\mathcal{Y}$. Observed values of the responses, represented by the vector $y$.

Predictors: $X\beta$, where the $n \times p$ matrix $X$ is the *model matrix*, $n$ is the number of obsrvations, and $p$ is the dimension of the *coefficient vector*, $\beta$. The coefficients are *parameters* in the model. We form *estimates* $\hat{\beta}$ of these paramters from the observed data. We assume that $n \geq q$.

$$\mathcal{Y} \sim \text{Multivariate Gaussian}(X\beta_T, \sigma^2I_n)$$

where $\beta_T$ is the "true", but unknown, value of the coefficient vector. The probability density of $\mathcal{Y}$ (is also called a *spherical normal density*): $$f_{\mathcal{Y}}(\mathbf{y}) = \frac{1}{(2\pi\sigma^{2})^{n/2}} \exp\left(\frac{-\parallel\mathbf{y} - \mathbf{X}\beta_{T}\parallel^{2}}{2\sigma^2}\right)$$

The likelihhod: $$L(\beta,\sigma | \mathbf{y}) = \frac{1}{(2\pi\sigma^{2})^{n/2}} \exp\left(\frac{-\parallel\mathbf{y} - \mathbf{X}\beta\parallel^{2}}{2\sigma^2}\right)$$

The *maximum likelihood estimates (mles)* of the parameters are the values of the parameters ($\hat{\beta}, \hat{\sigma}$) that maximize the likelihood.

The log-likelihood: $$\ell(\beta,\sigma|\mathbf{y})=\log(L(\beta,\sigma|\mathbf{y}))=-\frac{n}{2}\log(2\pi\sigma^{2})-\frac{-\parallel\mathbf{y}-\mathbf{X}\beta\parallel^{2}}{2\sigma^{2}}$$

The *deviance*: $$d(\beta,\sigma|\mathbf{y}) = -2\ell(\beta,\sigma|\mathbf{y}) = n\log(2\pi\sigma^{2}) + \frac{-\parallel\mathbf{y}-\mathbf{X}\beta\parallel^{2}}{\sigma^{2}}$$ 

Because of the negative sign, the mle’s are the values that minimize the deviance. For any fixed value of $\sigma^2$, the deviance is minimized with respect to $\beta$ when the residual sum of squares, $$S(\beta | \mathbf{y}) = \parallel\mathbf{y} - \mathbf{X}\beta\parallel^{2},$$  
is minimized. Thus the mle of the coefficient vector, $\hat{\beta}$, in the Gaussian linear model is the *least squares estimate* $$\hat{\beta}=\arg\,\min_{\beta}\,\parallel\mathbf{y}-\mathbf{X}\beta\parallel^2.$$


### Linear algebra of least squares
An *orthogonal* $n \times n$ matrix, $Q$, has the property $Q'Q=QQ'=I_{n}$. An orthogonal matrix has a special property that it **preserves lengths**.
$$\parallel Qx\parallel^{2}=(Qx)'Qx=x'Q'Qx=x'x=\parallel x\parallel^{2}$$

#### The QR decomposition
**Any** $n\times p$ matrix $X$ has a QR decomposition consisting of
an orthogonal $n\times n$ matrix $Q$ and a $p\times p$ matrix $R$
that is zero below the main diagonal (in other words, it is *upper
triangular*). The QR decomposition of the model matrix $X$ is written
$$X=Q\begin{bmatrix}R\\
0
\end{bmatrix}=\begin{bmatrix}Q_{1} & Q_{2}\end{bmatrix}\begin{bmatrix}R\\
0
\end{bmatrix}=Q_{1}R$$

where $Q_{1}$ is the first $p$ columns of $Q$ and $Q_{2}$ is the last $n-p$ columns of $Q$.

If the diagonal elements of $R$ are all non-zero (in practice this means that none of them are very small in absolute value) then $X$ has *full column rank* and the columns of $Q_1$ form an orthonormal basis for the column space of $X$ [col($X$)]. If the rank of $X$ is $k < p$ (rank-deficient), using a $p \times p$ permutation matrix $P$, we can make the first $k$ columns of $Q$ form an othonormal basis for col($XP$).

$$\begin{array}{cl}
\hat{\beta} & = \arg\,\min_{\beta}\,\parallel\mathbf{y}-\mathbf{X}\beta\parallel^2\\
 & = \arg\,\min_{\beta}\,\parallel Q' (\mathbf{y}-\mathbf{X}\beta)\parallel^2\\
 & = \arg\,\min_{\beta}\,\parallel Q'\mathbf{y}-Q'\mathbf{X}\beta\parallel^2\\
 & = \arg\,\min_{\beta}\,\parallel Q'\mathbf{y}-Q'QR\beta\parallel^2\\
 & = \arg\,\min_{\beta}\,\parallel Q_1'\mathbf{y}-R\beta\parallel^2 + \parallel Q_2'\mathbf{y}\parallel
\end{array} $$

If rank($X$) = $p$ then rank($R$) = $p$ and $R^{-1}$ exists so we can write $\hat{\beta} = R^{-1}Q_1'\mathbf{y}$, though you do not acutually calculate $R^{-1}$ to solve for $\hat{\beta}$.

In a model fit by the `lm()` or `aov()` functions in `R` there is a component `$effects` which is $Q'\mathbf{y}$. The component `$qr` is a condensed form of the $QR$ decomposition of the model matrix $X$. The matrix $R$ is embedded in there but the matrix $Q$ is a virtual matrix represented as a product of Householder reflections and not usually evaluated/created explicitly.

In [2]:
data(Formaldehyde)
Formaldehyde

,carb,optden
1,0.1,0.086
2,0.3,0.269
3,0.5,0.446
4,0.6,0.538
5,0.7,0.626
6,0.9,0.782


In [3]:
(X <- model.matrix(lm1 <- lm(optden ~ 1 + carb, Formaldehyde)))
# model.matrix returns X

,(Intercept),carb
1,1.0,0.1
2,1.0,0.3
3,1.0,0.5
4,1.0,0.6
5,1.0,0.7
6,1.0,0.9


In [4]:
# model.frame generates X and y
(y = model.response(model.frame(lm1)))

1     2     3     4     5     6 
0.086 0.269 0.446 0.538 0.626 0.782

In [5]:
class(qrlm1 <- lm1$qr)

[1] "qr"

In [6]:
(R <- qr.R(qrlm1))

,(Intercept),carb
1,-2.44949,-1.26557
2,0.0000000,0.6390097


In [7]:
(Q1 <- qr.Q(qrlm1))

-0.4082483,-0.6520507
-0.4082483,-0.3390663
-0.40824829,-0.02608203
-0.4082483,0.1304101
-0.4082483,0.2869023
-0.4082483,0.5998866


In [8]:
(Q1R <- Q1 %*% R) 

(Intercept),carb
1.0,0.1
1.0,0.3
1.0,0.5
1.0,0.6
1.0,0.7
1.0,0.9


In [9]:
all.equal(X, Q1R, check.attributes = F) # should be able to reconstruct model matrix X

[1] TRUE

In [10]:
(Q <- qr.Q(qrlm1, complete=TRUE)) # produce the full n*n orthogonal matrix Q

-0.4082483,-0.6520507,-0.3737045,-0.3405290,-0.3073534,-0.2410023
-0.40824829,-0.33906635,0.05460995,0.22071963,0.38682932,0.71904869
-0.40824829,-0.02608203,0.86857638,-0.14397908,-0.15653455,-0.18164549
-0.4082483,0.1304101,-0.1535966,0.8125532,-0.2212971,-0.2889976
-0.4082483,0.2869023,-0.1757696,-0.2309146,0.7139404,-0.3963496
-0.4082483,0.5998866,-0.2201156,-0.3178501,-0.4155847,0.3889463


In [11]:
as.vector(lm1$effects)

[1] -1.121458054  0.559955028  0.005139411  0.009915668  0.010691926
[6] -0.007755559

In [12]:
as.vector(crossprod(Q, y)) # crossprod(A, B) creates A'B directly without creating A' from A
# crosspod(X) creates X'X; 
# tcrossprod(X) creates XX'.

[1] -1.121458054  0.559955028  0.005139411  0.009915668  0.010691926
[6] -0.007755559

In [13]:
as.vector(qr.qty(qrlm1, y)) # another way to produce Q'y

[1] -1.121458054  0.559955028  0.005139411  0.009915668  0.010691926
[6] -0.007755559

In [14]:
zapsmall(crossprod(Q1)) # Q1'Q1 = I

1,0
0,1


In [15]:
zapsmall(crossprod(Q)) # Q'Q = I

1,0,0,0,0,0
0,1,0,0,0,0
0,0,1,0,0,0
0,0,0,1,0,0
0,0,0,0,1,0
0,0,0,0,0,1


Because the diagonal elements of $R$ are all safely non-zero, we can sove the system $R\hat{\beta} = Q_1'\mathbf{y}$ for the coefficient estimates, $\hat{\beta}$, using `backsolve()` or `solve()` function.

In [16]:
backsolve(R, crossprod(Q1, y))

0.005085714
0.8762857


In [17]:
coef(lm1)

(Intercept)        carb 
0.005085714 0.876285714

The function `qr.coef()` combines the multiplication of $y$ by $Q_1'$ and the `backsolve` step.

In [18]:
qr.coef(qrlm1, y)

(Intercept)        carb 
0.005085714 0.876285714

#### The determinant of an orthogonal matrix
Determinant of the $k \times k$ square matrix $A$ is the **volumn** of the parallelepiped spanned by its columns (or rows). Because we can consider either the rows or the columns when evaluating the determinant, we must have
$$|A| = |A'|$$

Furthermore, the determinant of a diagonal matrix or a triangular matrix is simply the product of its diagonal elements. 

the determinant, $|A|$ is evaluated in practice is by forming the $QR$ decomposition of $A$, taking the product of the diagonal elements of $R$, and determining whether $|Q|$ has a plus or a minus sign.

In [19]:
det(crossprod(Q))

[1] 1

In [26]:
tcrossprod(Q1) %*% y

0.09271429
0.2679714
0.4432286
0.5308571
0.6184857
0.7937429


In [32]:
qr.fitted(qrlm1, y)

1          2          3          4          5          6 
0.09271429 0.26797143 0.44322857 0.53085714 0.61848571 0.79374286

In [34]:
qr.resid(qrlm1, y); resid(lm1)

1            2            3            4            5            6 
-0.006714286  0.001028571  0.002771429  0.007142857  0.007514286 -0.011742857

1            2            3            4            5            6 
-0.006714286  0.001028571  0.002771429  0.007142857  0.007514286 -0.011742857

#### Comparison to the usual text-b o ok formulas

Most text books state that the least squares estimates are
$$\hat{\beta}=(X'X)^{-1}X'y$$
giving the impression that $\hat{\beta}$ is calculated this way.
It isn't.

If we substitute $X=Q_{1}R$ in the above equation, we get
$$(X'X)^{-1}X'y=(R'Q_{1}^{'}Q_{1}R)^{-1}R'Q_{1}^{'}y=(R'R)^{-1}R'Q_{1}^{'}y=R^{-1}(R')^{-1}R'Q_{1}^{'}y=R^{-1}Q_{1}^{'}y$$
our previous result.

**Whenever you see $X'X$ in a formula you should mentally replace it
by $R'R$ and similarly replace $(X'X)^{-1}$ by $R^{-1}(R')^{-1}$
then see if you can simplify the result.**

The R function `chol2inv()` calculates $(R'R)^{-1} = R^{-1}(R')^{-1}$ directly from
$R$ without evaluating $R^{-1}$ explicitly. In this way, it should
be faster and more accurate than evaluating $R^{-1}$ explicitly.

The determinant of $X'X$ is
$$|X'X|=|R'R|=|R|^{2}=\left(\prod_{i=1}^{p}r_{i,i}\right)^{2}$$


The fitted value $\hat{y}$ are $Q_{1}Q_{1}^{'}y$ and thus the *hat
matrix* (which puts a ``hat\textquotedblright{} on y by transforming
it to $\hat{y}$) is the $n\times n$ matrix $Q_{1}Q_{1}^{'}$. Often
we are interested in the diagonal elements of the hat matrix, which
are the sums of the squares of rows of $Q_{1}$. (In practice you
don\textquoteright t want to calculate the entire $n\times n$ hat
matrix just to get the diagonal elements when $n$ could be very large.)

The residuals, $\hat{e}=y-\hat{y}$, are calculated as $\hat{e}=Q_{2}Q_{2}^{'}y$.

The matrices $Q_{1}Q_{1}^{'}$ and $Q_{2}Q_{2}^{'}$ are *projection
matrices*, which means that they are symmetric and idempotent. ($A$
square matrix $A$ is idempotent if $AA=A$.) When $rank(X)=p$, the
hat matrix $Q_{1}Q_{1}^{'}$ projects any vector in $\mathbb{R}^{n}$
onto the column span of $X$. The other projection, $Q_{2}Q_{2}^{'}$,
is onto the subspace orthogonal to the column span of $X$.


#### The Cholesky decomposition
The Cholesky decomposition of a positive definite symmetric matrix,
which means a $p\times p$ symmetric matrix $A$ such that $x'Ax>0$
for all non-zero $x\in\mathbb{R}^{p}$ is of the form 

$$A=R'R=LL'$$

where $R$ is an upper triangular $p\times p$ matrix and $L=R'$
is lower triangular. The two forms are the same decomposition: it
is just a matter of whether you want $R$, the factor on the right,
or $L$, the factor on the left. Generally statisticians write the
decomposition as $R'R$. 

The decomposition is only determined up to changes in sign of the
rows of $R$ (or, equivalently, the columns of $L$). For definiteness
we require positive diagonal elements in $R$. When $\text{rank}(X)=p$
the Cholesky decomposition $R$ of $X'X$ is the equal to the matrix
$R$ from the $QR$ decomposition up to changes in sign of rows. The
matrix $X'X$ matrix is obviously symmetric and it is positive definite
because $x'(X'X)x=x'(R'R)x=||Rx||^{2}\geq0$ with equality only when
$Rx=0$, which, when $\text{rank}(X)=p$, implies that $x=0$.

The R function `chol()` evaluate the Cholesky decomposition; `chol2inv()` creates $(X'X)^{-1}$ directly from the Cholesky decomposition
of $X'X$. **Generally the QR decomposition is preferred to the Cholesky
decomposition for least squares problems** because there is a certain
loss of precision when forming $X'X$. However, when $n$ is very
large you may want to build up $X'X$ using blocks of rows. Also,
if $X$ is sparse it is an advantage to use sparse matrix techniques
to evaluate and store the Cholesky decomposition.



In [37]:
chol(crossprod(X))

,(Intercept),carb
(Intercept),2.44949,1.26557
carb,0.0000000,0.6390097
